In [1]:
import sys
sys.path.append("..")

In [2]:
#from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *
from squlearn.kernel import *

We will show that the kernel derivatives calculated by squlearn are the same as the ones analytically calculated for the separable rx fmap

In [3]:
from circuits.circuits import Separable_rx

Same K(X, X) 1D

In [4]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

#create x_vec a tensor with 1, 2 , 3
x_vec = sp.IndexedBase("x")
y_vec = sp.IndexedBase("y")
 
def k_(x_vec, y_vec, n):
    s = 0
    for i in range(0, n):
        s+= 2 - 2*sp.cos(x_vec[i]-y_vec[i])
    total = sp.exp(-gamma_sp*s)
    return total

def substitute_manually(obj, x, y, gamma_num):
    dictionary_to_sub = {}
    dictionary_to_sub[gamma_sp] = gamma_num
    for i in range(len(x)):
        dictionary_to_sub[x_vec[i]] = x[i]
        dictionary_to_sub[y_vec[i]] = y[i]
    return obj.subs(dictionary_to_sub).evalf()
    

    

In [19]:
x_train = np.array([[0, 1],[1, .5]])
y_train = np.array([[1, 1.2, 7],[0, 1.2, 1]])
x_train = np.array([[0, 1.2, 4],[1, 1.2, 2]])

x_train = np.array([[0, 1.5, 0.3],[1, .5, 0.7]])
x_train = np.array([[0], [.5]])

y_train= x_train

n_feature = x_train.shape[1]
feature_map = Separable_rx(n_feature,1,n_feature)
q_kernel = ProjectedQuantumKernel(feature_map, Executor())

In [20]:
def eval_K_matrix_analytically(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset)))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            num_matrix[i,j] = substitute_manually(k_(x_vec, y_vec, num_dimensions), x_dataset[i], y_dataset[j], 1)
    return num_matrix

def eval_K_matrix_analytically_dx(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset), len(x_dataset[0])))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            for l in range(len(x_dataset[0])):
                num_matrix[i,j, l] = substitute_manually(k_(x_vec, y_vec, num_dimensions).diff(x_vec[l]), x_dataset[i], y_dataset[j], 1)
    return num_matrix

def eval_K_matrix_analytically_dxdx(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset), len(x_dataset[0])))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            for l in range(len(x_dataset[0])):
                num_matrix[i,j, l] = substitute_manually(k_(x_vec, y_vec, num_dimensions).diff(x_vec[l]), x_dataset[i], y_dataset[j], 1)
    return num_matrix

def eval_K_matrix_analytically_dy(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset), len(x_dataset[0])))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            for l in range(len(x_dataset[0])):
                num_matrix[i,j, l] = substitute_manually(k_(x_vec, y_vec, num_dimensions).diff(y_vec[l]), x_dataset[i], y_dataset[j], 1)
    return num_matrix

In [21]:
q_kernel.evaluate(x_train, y_train)

array([[1.        , 0.78283379],
       [0.78283379, 1.        ]])

In [22]:
eval_K_matrix_analytically(x_train, y_train)

array([[1.        , 0.78283379],
       [0.78283379, 1.        ]])

In [27]:
q_kernel.evaluate_derivatives(x_train, y_train, ["dKdxdx"])

{'x': array([[0. ],
        [0.5]]),
 'param': array([], dtype=float64),
 'param_op': array([], dtype=float64),
 'dKdx': array([[ 0.        ,  0.75062102],
        [-0.75062102,  0.        ]]),
 'dKdxdx': array([[-2.        , -0.65426879],
        [-0.65426879, -2.        ]])}

In [10]:
eval_K_matrix_analytically_dy(x_train, y_train)

array([[[ 0.        ,  0.        ,  0.        ],
        [-0.22852141,  0.22852141, -0.10575579]],

       [[ 0.22852141, -0.22852141,  0.10575579],
        [ 0.        ,  0.        ,  0.        ]]])

In [19]:
from itertools import combinations
list(combinations(range(4), 2))

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [28]:
vanilla_rbf = sp.exp(-gamma_sp*(x-y)**2)

In [30]:
vanilla_rbf.diff(y)

-gamma*(-2*x + 2*y)*exp(-gamma*(x - y)**2)

In [29]:
vanilla_rbf.diff(x)

-gamma*(2*x - 2*y)*exp(-gamma*(x - y)**2)

In [22]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
x3, y3 = sp.symbols('x_3 y_3')
p1, p2 = sp.symbols('p_1 p_2')
O1 = sp.symbols('O_1', cls=sp.Function)
O2 = sp.symbols('O_2', cls=sp.Function)
O3 = sp.symbols('O_3', cls=sp.Function)


In [23]:
k = sp.symbols('k', cls=sp.Function)
pqk_fun =  k(O1(x, p1, p2), O2(x, p1, p2), O3(x, p1, p2), O1(y, p1, p2), O2(y,p1, p2), O3(y,p1, p2))
#pqk_fun =  k(O1(x, p1, p2), O2(x, p1, p2), O1(y, p1, p2), O2(y,p1, p2))
pqk_fun_dx = sp.diff(pqk_fun, x).simplify()
pqk_fun_dxdx = sp.diff(pqk_fun_dx, x).simplify()

In [24]:
pqk_fun_dx

Derivative(O_1(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_1(x, p_1, p_2)) + Derivative(O_2(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_2(x, p_1, p_2)) + Derivative(O_3(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_3(x, p_1, p_2))

In [25]:
pqk_fun_dx.diff(x)

(Derivative(O_1(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), (O_1(x, p_1, p_2), 2)) + Derivative(O_2(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_1(x, p_1, p_2), O_2(x, p_1, p_2)) + Derivative(O_3(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_1(x, p_1, p_2), O_3(x, p_1, p_2)))*Derivative(O_1(x, p_1, p_2), x) + (Derivative(O_1(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), O_1(x, p_1, p_2), O_2(x, p_1, p_2)) + Derivative(O_2(x, p_1, p_2), x)*Derivative(k(O_1(x, p_1, p_2), O_2(x, p_1, p_2), O_3(x, p_1, p_2), O_1(y, p_1, p_2), O_2(y, p_1, p_2), O_3(y, p_1, p_2)), (O_2(x, p_1, p_2), 2)) + Derivative(O_3(x, p_1, p_2), x)

Fundamentally, we implement the below equation using einsum notation

In [ ]:
pqk_fun_dxdx.simplify()

Derivative(O_1(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2)) + 2*Derivative(O_1(x), x)*Derivative(O_2(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_2(x)) + 2*Derivative(O_1(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_3(x)) + Derivative(O_1(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x)) + Derivative(O_2(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_2(x), 2)) + 2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x), O_3(x)) + Derivative(O_2(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x)) + Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2)) + Derivative(O_3(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_3(x))

In [ ]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [ ]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [ ]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.       , -0.4298443],
       [ 0.4298443, -0.       ]])

In [ ]:
K_dfdx

array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [ ]:
K_separable_rx_PQK_dxdx(X,X,gamma)

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [ ]:
K_dfdxdx

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [ ]:
import numpy as np
#from sklearn import RBF
from sklearn.gaussian_process.kernels import RBF

In [ ]:
from circuits.circuits import Separable_rx

In [ ]:
sigma=1
gamma= 1/(2*sigma**2)
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1, num_features=1), executor=Executor("pennylane"), gamma=gamma)

In [ ]:
X = [0.1, -0.41]

In [ ]:
pqk_squlearn.evaluate_derivatives(X, X, "dfdx")

Calculating dfdx
Calculating f
Calculating f


array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [ ]:
pqk_squlearn.evaluate_derivatives(X, X, "dfdxdx")

Calculating dfdx
Calculating dfdxdx
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f


array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])